In [1]:
import pandas as pd
import numpy as np

In [2]:
ar = pd.read_csv('/Users/meredithmay/Downloads/arb.csv')
al = pd.read_csv('/Users/meredithmay/Downloads/arl.csv')
indy = pd.read_csv('/Users/meredithmay/Downloads/indy.csv')
gr = pd.read_csv('/Users/meredithmay/Downloads/gr.csv')

In [3]:
whole = ar.merge(al,on = ['uid_causing_drop','error_reason','sw_version','drop_health_level'],how = 'outer',suffixes = ['_ar','_al'])
whole = whole.merge(indy,on = ['uid_causing_drop','error_reason','sw_version','drop_health_level'],how = 'outer')
whole = whole.merge(gr,on = ['uid_causing_drop','error_reason','sw_version','drop_health_level'],how = 'outer')


In [4]:
whole = whole.fillna(0)

In [5]:
whole['number_of_occurance'] = whole.number_of_occurance_ar + whole.number_of_occurance_al + whole.number_of_occurance_x + whole.number_of_occurance_y

In [6]:
save = whole.drop(columns = [ 'number_of_occurance_ar', 'number_of_occurance_al', 'number_of_occurance_x', 'number_of_occurance_y'])

In [9]:
save.to_csv('/Users/meredithmay/Downloads/wholesite.csv')

## SQL 

with cte as
(
	select v.git_branch, h.*
	from health_drops h
	join vehicle_logs v on v.log_name = h.log_name
	where v.git_branch in ('release/7.0','release/7.1','release/7.3')
	and v.is_prod = true
    and h.pre_run_level = 7

)

select uid_causing_drop, error_reason, sw_version, drop_health_level, count(*) as number_of_occurance
from
(select unnest(error_uid) as uid_causing_drop, unnest(error_uid_description) as error_reason, drop_health_level,
 git_branch as sw_version
from cte) t
group by uid_causing_drop, error_reason,sw_version, drop_health_level
order by uid_causing_drop, error_reason,sw_version, number_of_occurance desc, drop_health_level desc

create view meredith_health_drop_count_arb as
(
with cte as
(
	select v.git_branch, h.*
	from health_drops h
	join vehicle_logs v on v.log_name = h.log_name
	where v.git_branch in ('release/7.0','release/7.1','release/7.3')
	and v.is_prod = true
    and h.pre_run_level = 7)
select uid_causing_drop, error_reason, sw_version, drop_health_level, count(*) as number_of_occurance
from
(select unnest(error_uid) as uid_causing_drop, unnest(error_uid_description) as error_reason, drop_health_level,
 git_branch as sw_version
from cte) t
group by uid_causing_drop, error_reason,sw_version, drop_health_level
order by uid_causing_drop, error_reason,sw_version, number_of_occurance desc, drop_health_level desc)